In [108]:
# Try linear regression, logistic regression SVM, decision trees 
# then jaccard index, cosine similarity, weighted kernel density estimator
#
# regularlize all models -- later. I have a lot of data.
# train, cross validate  -- later. I need some predictive power.
#
# start with tech, then for all entries in my database 
# what predictors did greenberg use?
#
# duration Length of project in Days Double
# twitter_url Connected to twitter Boolean
# has_video Video present Boolean
# facebook_connected Connected to Facebook Boolean
# facebook_friends Number of facebook friends Integer
# twitter_followers Number of twitter followers Integer
# sent Sentiment (pos, neg, or neutral) String --
#      For the sentiment attribute, we used the Mashape Text-Processing API
# fkgl Grade level Double -FKGL stands for Flesch-Kincaid Grade Level (measures reading ability) 
#       from nltk
#sent_count Number of sentences in project description Integer
# see Tindle blog post


In [2]:
# Features I already have _______________________________________
# - rewards: n, min, max, intn shipping
# - n creator backed
# - title char len
# - blurb char len
# - launch time (in Unix time)
# - project duration
# - goal

# I could add easily 
# - day of the month
# - month
# - time between launch and ship of rewards
# - title word len
# - blurb word len

In [3]:
# After modeling, 
# Other possible features to extract _____________________________
# - n pictures -- use beautiful soup
# - has video
# - project body length 
# - project n spelling errors
# - text quality, sentiment analysis
# - other reward parameters

# lower priority -- 
# - international projects


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
#import seaborn as sns

In [5]:
%matplotlib qt

In [6]:
#from extract_kickstarter_features import *
#%load_ext autoreload
#%autoreload 2

In [7]:
saveas_path = '/Users/jamie/insight data/figures/'

In [75]:
tech_df = pd.read_pickle('tech_df.save')

In [76]:
np.shape(tech_df)

(6633, 12)

In [77]:
tech_df.tail(3)

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur
6630,16468,8904,5000,7,1,100,1,0,56,129,1411686079,3459600
6631,16469,615,50000,8,1,1000,0,0,51,103,1409441823,5184000
6632,16470,585549,7500,11,1,1337,1,2,44,114,1413159586,2595600


In [78]:
# create new columns 
tech_df['pledged/goal'] = tech_df['pledged']/tech_df['goal']

In [79]:
tech_df.tail(3)

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur,pledged/goal
6630,16468,8904,5000,7,1,100,1,0,56,129,1411686079,3459600,1.7808
6631,16469,615,50000,8,1,1000,0,0,51,103,1409441823,5184000,0.0123
6632,16470,585549,7500,11,1,1337,1,2,44,114,1413159586,2595600,78.0732


In [13]:
#let's find some data stories!

In [13]:
# restrict to projects with goals of less than $500,000
reasonable_goal_is = np.where(tech_df['goal'] < 500000)[0]
print np.shape(reasonable_goal_is)

(6546,)


In [91]:
# restrict to projects with goals of less than $50,000
small_goal_is = np.where(tech_df['goal'] < 25000)[0]
print np.shape(small_goal_is)

(3886,)


In [92]:
# Create linear regression object
regr = linear_model.LinearRegression(normalize=False)

In [93]:
predictor_cols = list(['n rewards','min reward','max reward','intn ship','n creator backed','title n chars','blurb n chars','campaign dur'])
predictors = tech_df.ix[small_goal_is,predictor_cols]

In [94]:
pledged = tech_df.ix[small_goal_is,'pledged']

In [95]:
regr.fit(predictors,pledged)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [96]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(predictors) - pledged) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(predictors, pledged))

('Coefficients: \n', array([  1.78097753e+03,   1.50094022e+00,  -5.35587887e-01,
         3.10800095e+03,   1.01171885e+02,   1.05225233e+02,
        -8.84521957e+00,   2.95908945e-04]))
Residual sum of squares: 991760638.34
Variance score: 0.08


In [80]:
# now try a logistic regression ___________
tech_df.ix[:,'outcomes'] = 0

In [81]:
tech_df.ix[tech_df['pledged'] >= tech_df['goal'],'outcomes'] = 1

In [82]:
tech_df.tail(5)

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur,pledged/goal,outcomes
6628,16461,25,100000,5,25,10000,-1,0,35,134,1414879251,2595600,0.00025,0
6629,16462,15790,3500,8,59,250,1,1,36,132,1411511254,2592000,4.511429,1
6630,16468,8904,5000,7,1,100,1,0,56,129,1411686079,3459600,1.7808,1
6631,16469,615,50000,8,1,1000,0,0,51,103,1409441823,5184000,0.0123,0
6632,16470,585549,7500,11,1,1337,1,2,44,114,1413159586,2595600,78.0732,1


In [102]:
clf = linear_model.LogisticRegression(C=.15)
clf.fit(predictors,tech_df.ix[small_goal_is,'outcomes'])

LogisticRegression(C=0.15, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [103]:
clf.coef_

array([[  4.16002478e-07,  -4.62030800e-06,   1.81356642e-05,
          4.78514531e-08,   7.66173901e-07,   3.01870794e-07,
         -8.79569759e-07,  -2.38083563e-07]])

In [105]:
# next try an svm, random forests. I'll need to scale it. 
# http://scikit-learn.org/stable/modules/svm.html
# http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing

In [107]:
# I think I need more predictors, nlp, and a distribution 
# latent variable analysis?